In [1]:
import pandas as pd
# from utils import select_min_pay, read_input_file, export_file
import glob
import numpy as np
import pandas as pd
import glob
import logging
import win32com.client as win32
import os
import jaydebeapi
from const import QUERY, CONNECT

In [14]:
import oracledb

try:
    # Provide a full connection string
    connection = oracledb.connect(
        user="supat",
        password="SU#pass005",
        dsn="(DESCRIPTION=(ADDRESS_LIST=(ADDRESS=(PROTOCOL=TCP)(HOST=10.1.41.100)(PORT=1622)))(CONNECT_DATA=(SID=your_SID_here)))"
    )
    print("Connection successful!")
except oracledb.Error as e:
    print(f"Error: {e}")

Error: DPY-6005: cannot connect to database (CONNECTION_ID=AxhIyW4ax5R3P1mLLOpTug==).
DPY-6003: SID "your_SID_here" is not registered with the listener at host "10.1.41.100" port 1622. (Similar to ORA-12505)


In [ ]:

conn_str = u'supat/SU#pass005@10.1.41.100:1622/JFDWH'
conn = cx_Oracle.connect(conn_str)
c = conn.cursor()

ModuleNotFoundError: No module named 'cx_Oracle'

In [5]:
!pip install cx_Oracle

In [15]:
JDBC_URL = "jdbc:oracle:thin:@//10.1.41.100:1622/JFDWH"  # Update details
USER = "supat"
PASSWORD = "SU#pass005"
DRIVER_CLASS = "oracle.jdbc.OracleDriver"


# Establish connection
conn = jaydebeapi.connect(
    DRIVER_CLASS,
    JDBC_URL,
    [USER, PASSWORD],
)

print("Connected to Oracle Database!")


TypeError: Class oracle.jdbc.OracleDriver is not found

In [6]:
def connectdate():
    try:
        # Establish connection to the database
        print(CONNECT.DB)
        conn = jaydebeapi.connect(
            CONNECT.DB,
            CONNECT.PORT,
            CONNECT.USER,
            CONNECT.CNN,
        )
        print("Connection established successfully!")

        # Create a cursor to interact with the database
        # cur = conn.cursor()

        # Execute the query and retrieve the result
        print("startquery")
        sql = f'''
   WITH rawdata AS (
SELECT tcc.CONTRACT_NO ,tcc.NATIONAL_ID ,tcc.GROSS_INCOME ,tcc.OCCUPATION_TYPE ,
tcc.CONTRACT_DATE ,floor(MONTHS_BETWEEN(SYSDATE, tcc.CONTRACT_DATE))  AS contract_period,
FLOOR(MONTHS_BETWEEN(SYSDATE, tcc.DATE_OF_BIRTH) / 12) AS age,
PRINCIPAL_BAL ,
CASE WHEN floor(MONTHS_BETWEEN(SYSDATE, tcc.CONTRACT_DATE))  < 7 THEN '06' ELSE '07'END AS MOB,
CASE 
        WHEN tcc.GROSS_INCOME BETWEEN 0 AND 15000 THEN '3' 
        WHEN tcc.GROSS_INCOME BETWEEN 15001 AND 30000 THEN '3'
        WHEN tcc.GROSS_INCOME BETWEEN 30001 AND 50000 THEN '2'
        WHEN tcc.GROSS_INCOME BETWEEN 50001 AND 100000 THEN '1'
        WHEN tcc.GROSS_INCOME > 100000 THEN '1'
        ELSE NULL 
    END AS  digit2income,
    tas.INDEX_SCORE AS digit2age
FROM TEMP_CONTRACT_CUSTOMER tcc 
LEFT JOIN TEMP_AGE_SCORE tas ON tas.AGE = FLOOR(MONTHS_BETWEEN(SYSDATE, tcc.DATE_OF_BIRTH) / 12) 
)
SELECT t2.*,
t3.score AS digit2_totalscore,
 'L' || t3.SCORE || t2.MOB AS final_score
FROM  rawdata t2
LEFT JOIN TEMP_DIGIT2_SCORE t3 ON t3.sumdigit = t2.digit2income+t2.digit2age
    '''
        cur = conn.cursor()
        cur.execute(sql)
        print("Execute")

        # Fetch and convert results to DataFrame
        rows = cur.fetchall()
        col_names = [desc[0] for desc in cur.description]
        cur.close()
        print("close")

        min_pay = pd.DataFrame(rows, columns=col_names)
        print("finishqueryquery")

        return min_pay

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

    # finally:
    #     # Ensure the connection is closed even if an error occurs
    #     if conn:
    #         conn.close()
    #         print("Connection closed.")

In [7]:
connectdate()

Error occurred: name 'CONNECT' is not defined
